Generate OWL ontology from VoID profile

TODO: this way? But VoID is missing data properties like `rdfs:label`

```turtle
up:Gene rdfs:label xsd:string ;
  up:encode up:Protein .
```

In [1]:
import requests
from rdflib import Graph

query = """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

CONSTRUCT {
    ?class1 a owl:Class ;
        rdfs:label ?class1Label ;
        rdfs:comment ?class1Comment ;
        rdfs:subClassOf ?superClass1 .

    ?prop a owl:ObjectProperty ;
        rdfs:domain ?class1 ;
        rdfs:range ?class2 ;
        rdfs:label ?propLabel ;
        rdfs:comment ?propComment ;
        rdfs:subPropertyOf ?superProp .

    ?class2 a owl:Class ;
        rdfs:label ?class2Label ;
        rdfs:comment ?class2Comment .

} WHERE {
    ?s <http://www.w3.org/ns/sparql-service-description#graph> ?graph .
    ?graph void:classPartition ?cp1 .
    ?cp1 void:class ?class1 ;
          void:propertyPartition ?pp1 .
    ?pp1 void:property ?prop ;
          void:triples ?pp1triples ;
          void:classPartition ?cp2 .
    ?cp2 void:class ?class2 .

    OPTIONAL { ?class1 rdfs:label ?class1Label . }
    OPTIONAL { ?class2 rdfs:label ?class2Label . }
    OPTIONAL { ?prop rdfs:label ?propLabel . }

    OPTIONAL { ?class1 rdfs:comment ?class1Comment . }
    OPTIONAL { ?class2 rdfs:comment ?class2Comment . }
    OPTIONAL { ?prop rdfs:comment ?propComment . }

    OPTIONAL {
        ?class1 rdfs:subClassOf ?superClass1 .
        FILTER (isIRI(?superClass1))
    }
    OPTIONAL {
        ?prop rdfs:subPropertyOf ?superProp .
        FILTER (isIRI(?superProp))
    }
    OPTIONAL {
        ?class2 rdfs:subClassOf ?superClass2 .
        FILTER (isIRI(?superClass2))
    }
} ORDER BY DESC(?pp1triples)"""

response = requests.post(
    "https://sparql.uniprot.org/sparql/",
    headers={"Accept": "text/turtle"},
    data={"query": query},
    timeout=60,
)
response.raise_for_status()
g = Graph()
g.bind("faldo", "http://biohackathon.org/resource/faldo#")
g.bind("up", "http://purl.uniprot.org/core/")

g.parse(data=response.text, format="turtle")

# print(response.text)
print(g.serialize(format="turtle"))
print(len(g))

@prefix faldo: <http://biohackathon.org/resource/faldo#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix up: <http://purl.uniprot.org/core/> .

faldo:begin a owl:ObjectProperty ;
    rdfs:domain faldo:InRangePosition,
        faldo:Position,
        faldo:Region ;
    rdfs:range faldo:ExactPosition,
        faldo:InRangePosition,
        faldo:OneOfPosition,
        faldo:Position .

faldo:end a owl:ObjectProperty ;
    rdfs:domain faldo:InRangePosition,
        faldo:Position,
        faldo:Region ;
    rdfs:range faldo:ExactPosition,
        faldo:InRangePosition,
        faldo:OneOfPosition,
        faldo:Position .

faldo:possiblePosition a owl:ObjectProperty ;
    rdfs:domain faldo:OneOfPosition,
        faldo:Posi

In [15]:
import requests
from rdflib import Graph

# TODO: build decent SHACL shapes?

query = """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX void: <http://rdfs.org/ns/void#>

CONSTRUCT {
    ?classShape a sh:NodeShape ;
                sh:targetClass ?class1 ;
                sh:property ?propertyShape ;
                rdfs:label ?class1Label .

    ?propertyShape sh:path ?prop ;
                   sh:class ?class2 ;
                   sh:description ?propLabel ;
                   sh:node ?class2Shape ;
                   sh:minCount 0 ;
                   sh:maxCount ?maxCount .

    ?class2Shape a sh:NodeShape ;
                 sh:targetClass ?class2 ;
                 rdfs:label ?class2Label .
} WHERE {
    ?s <http://www.w3.org/ns/sparql-service-description#graph> ?graph .
    ?graph void:classPartition ?cp1 .
    ?cp1 void:class ?class1 ;
          void:propertyPartition ?pp1 .
    ?pp1 void:property ?prop ;
          void:triples ?pp1triples ;
          void:classPartition ?cp2 .
    ?cp2 void:class ?class2 .

    OPTIONAL { ?class1 rdfs:label ?class1Label . }
    OPTIONAL { ?class2 rdfs:label ?class2Label . }
    OPTIONAL { ?prop rdfs:label ?propLabel . }

    # Generate unique shapes for classes and properties
    BIND(IRI(CONCAT(STR(?class1), "_Shape")) AS ?classShape)
    BIND(IRI(CONCAT(STR(?class2), "_Shape")) AS ?class2Shape)
    BIND(IRI(CONCAT(STR(?class1), "/property/", STRAFTER(STR(?prop), "#"))) AS ?propertyShape)

    # For demonstration, setting maxCount as a static example. It can be adjusted based on the actual data or requirements
    BIND(1 AS ?maxCount)
} ORDER BY DESC(?pp1triples)"""

response = requests.post(
    "https://sparql.uniprot.org/sparql/",
    headers={"Accept": "text/turtle"},
    data={"query": query},
    timeout=60,
)
response.raise_for_status()
g = Graph()
g.bind("faldo", "http://biohackathon.org/resource/faldo#")
g.bind("up", "http://purl.uniprot.org/core/")

g.parse(data=response.text.replace("[ ],", ""), format="turtle")
# NOTE: replace is a hack to remove useless blank nodes

# print(response.text)
print(g.serialize(format="turtle"))
print(len(g))

@prefix faldo: <http://biohackathon.org/resource/faldo#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix up: <http://purl.uniprot.org/core/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

up:Database_Shape a sh:NodeShape ;
    rdfs:label "Database (description of)" ;
    sh:property <http://purl.uniprot.org/core/Database/property/> ;
    sh:targetClass up:Database .

up:Mass_Measurement_Method_Shape a sh:NodeShape ;
    rdfs:label "Mass Measurement" ;
    sh:property <http://purl.uniprot.org/core/Mass_Measurement_Method/property/isDefinedBy> ;
    sh:targetClass up:Mass_Measurement_Method .

up:Organelle_Shape a sh:NodeShape ;
    rdfs:label "Organelle" ;
    sh:property <http://purl.uniprot.org/core/Organelle/pr